Preparing to submit wold stranded samples....

This got a little messy because a couple libraries needed a top up.

In [1]:
import os
import sys
import requests
import pandas
import paramiko
import json
from IPython import display
from pathlib import Path

In [2]:
from curation_common import *
from htsworkflow.submission.encoded import DCCValidator

In [3]:
from htsworkflow.submission.encoded import Document
from htsworkflow.submission.aws_submission import run_aws_cp

In [4]:
GCATDIR = os.path.expanduser('~/src/gcat')
if GCATDIR not in sys.path:
    sys.path.append(GCATDIR)
    import gcat

In [5]:
# live server & control file
server = ENCODED('www.encodeproject.org')
spreadsheet_name = '10X Wold Submission objects'
engine=None
#engine='odf'

# test server & datafile
#server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-encode3-limb-2017-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [6]:
award = 'UM1HG009443'

# Submit Documents

In [7]:
cellranger_uuid = '/documents/aca954d8-6133-428c-8b4a-2172dfa4066b/'
cellranger = Document(
    os.path.expanduser('~/woldlab/ENCODE/cellranger_mkref.pdf'),
    'pipeline protocol',
    'Cell Ranger index building',
)
body = cellranger.create_if_needed(server, cellranger_uuid, validator)
if '@graph' in body:
    print(body['@graph'][0]['@id']) 
else:
    print(body['@id'])

/documents/aca954d8-6133-428c-8b4a-2172dfa4066b/


# Register Reference

In [25]:
book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
references = book.parse('Reference', header=0)
created = server.post_sheet('/references/', 
                            references, 
                            verbose=True, 
                            dry_run=False,  
                            validator=validator)
print(len(created))

Reponse {'status': 'success', '@type': ['result'], '@graph': [{'schema_version': '19', 'related_files': [], 'documents': ['/documents/2297237f-dfb0-424d-a386-c350ffe8dbe3/'], 'accession': 'ENCSR125TRU', 'alternate_accessions': [], 'description': 'cellranger indexed reference files with the ERCC spikeins', 'dbxrefs': [], 'internal_tags': [], 'status': 'in progress', 'references': [], 'date_created': '2020-11-04T19:06:41.484665+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'lab': '/labs/barbara-wold/', 'award': '/awards/UM1HG009443/', 'aliases': [], 'reference_type': 'index', 'organism': '/organisms/mouse/', 'software_used': ['/software-versions/69416c05-7689-4846-9ad4-b1bacc15e8a5/'], '@id': '/references/ENCSR125TRU/', '@type': ['Reference', 'FileSet', 'Dataset', 'Item'], 'uuid': 'dbf6997f-ff4d-4f46-81a7-79253205eb88', 'original_files': [], 'contributing_files': [], 'files': [], 'revoked_files': [], 'assembly': []}]}
row 0 created: ENCSR125TRU
1


In [28]:
created[0]['@id'], created[0]['uuid']

('/references/ENCSR125TRU/', 'dbf6997f-ff4d-4f46-81a7-79253205eb88')

# Register Libraries

In [ ]:
print(spreadsheet_name)
libraries = pandas.read_excel(spreadsheet_name, sheet_name='Library', header=0, engine=engine)
created = server.post_sheet('/libraries/', 
                            libraries, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

In [ ]:
if created:
    libraries.to_excel('/dev/shm/libraries.xlsx', index=False)

# Register Experiments

In [ ]:
print(server.server)
experiments = pandas.read_excel(spreadsheet_name, sheet_name='Experiment', header=0, engine=engine)
experiments = experiments[experiments['accession'] != 'barbara approval needed']
created = server.post_sheet('/experiments/', 
                            experiments, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

In [ ]:
if created:
    experiments.to_excel('/dev/shm/experiments.xlsx', index=False)

# Register Replicates

# Check Reference Files

In [36]:
book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
files = book.parse('ReferenceFile', header=0)
created = server.post_sheet('/files/', files, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

1
